# Blockchain Audit Prototype
## BizClear - Immutable Audit Trail Demo

**Why blockchain:** Immutability guarantees for permit and inspection records. Cryptographic hashes ensure tamper detection.

**Prerequisites:** Run `docker-compose up -d ganache` and deploy contracts (or `docker-compose up` for full stack).

**RPC:** Ganache at localhost:7545 (host) or localhost:8545 (if running notebook inside Docker).

## 1. Environment Setup

Connect to the blockchain. Checks if we're connected and shows the latest block.

In [ ]:
import json
from pathlib import Path
from web3 import Web3

RPC_URL = 'http://127.0.0.1:7545'
BUILD_DIR = Path('../build/contracts')

w3 = Web3(Web3.HTTPProvider(RPC_URL))
print(f'Connected: {w3.is_connected()}')
print(f'Block number: {w3.eth.block_number}')

## 2. Load Contract ABIs and Addresses

Load the AuditLog contract so we can talk to it. (Gets the contract info from the build folder.)

In [ ]:
import os, re

def _camel_to_snake(name):
    """AuditLog -> AUDIT_LOG, DocumentStorage -> DOCUMENT_STORAGE"""
    return re.sub(r'(?<=[a-z0-9])(?=[A-Z])', '_', name).upper()

def load_contract(name):
    """Load contract ABI from build dir; get address from .env first, then build artifact."""
    path = BUILD_DIR / f'{name}.json'
    with open(path) as f:
        data = json.load(f)
    abi = data['abi']

    # 1) Prefer address from .env (deploy-contracts keeps this current across Ganache restarts)
    #    Try multiple key formats: AUDIT_LOG_CONTRACT_ADDRESS, AUDITLOG_CONTRACT_ADDRESS
    snake = _camel_to_snake(name)  # AuditLog -> AUDIT_LOG
    addr = (os.getenv(f'{snake}_CONTRACT_ADDRESS')
            or os.getenv(f'{name.upper()}_CONTRACT_ADDRESS'))

    # 2) Fall back to build artifact (may be stale after Ganache restart)
    if not addr:
        networks = data.get('networks', {})
        addr = next((n.get('address') for n in networks.values() if n.get('address')), None)

    if not addr:
        raise FileNotFoundError(f'No deployed address for {name}. Run: docker-compose up deploy-contracts')

    addr = Web3.to_checksum_address(addr)

    # Sanity check: is there actually code at this address?
    if w3.eth.get_code(addr) == b'':
        raise RuntimeError(
            f'No contract code at {addr} for {name}. '
            'Ganache may have restarted. Run: docker-compose up deploy-contracts'
        )

    return w3.eth.contract(address=addr, abi=abi)

# Load .env so contract addresses are available
try:
    from dotenv import load_dotenv
    load_dotenv(Path().resolve().parent.parent / '.env', override=True)
except (ImportError, Exception):
    pass

audit_log = load_contract('AuditLog')
print('AuditLog loaded at', audit_log.address)

# Also load AccessControl so we can check roles
access_control = load_contract('AccessControl')
print('AccessControl loaded at', access_control.address)

## 3. Audit Trail Demo - Log Hash

Turn sample data into a hash and log it on-chain. Uses the deployer account (has AUDITOR_ROLE) — same key as audit-service, no setup needed.

In [ ]:
import hashlib
from eth_account import Account

# Read deployer key from .env (no hardcoded fallback — avoids key leakage if notebook is shared)
DEPLOYER_KEY = os.getenv('DEPLOYER_PRIVATE_KEY')
if not DEPLOYER_KEY:
    raise RuntimeError(
        'DEPLOYER_PRIVATE_KEY not set. Add it to .env (Ganache first account from mnemonic).\n'
        'Run: docker-compose up  (deploy-contracts writes it to .env automatically)'
    )
account = Account.from_key(DEPLOYER_KEY)

if w3.eth.get_balance(account.address) == 0:
    raise RuntimeError(f'Account {account.address} has no ETH. Is Ganache running? Is the key correct?')

# Check AUDITOR_ROLE on AccessControl
try:
    auditor_role = access_control.functions.AUDITOR_ROLE().call()
    has_role = access_control.functions.hasRole(account.address, auditor_role).call()
    ac_owner = access_control.functions.owner().call()
    print(f'AccessControl owner: {ac_owner}')
    print(f'Account has AUDITOR_ROLE: {has_role}')
    if not has_role:
        print(f'⚠ Account {account.address} does NOT have AUDITOR_ROLE!')
        print(f'  AccessControl owner is {ac_owner} — that account has the role.')
        print(f'  Fix: either use the owner key, or grant AUDITOR_ROLE to this account.')
except Exception as e:
    print(f'Could not check AUDITOR_ROLE: {e}')

print(f'Using account: {account.address} (balance: {w3.from_wei(w3.eth.get_balance(account.address), "ether"):.2f} ETH)')

event_data = 'permit_application_submitted_2024'
h = hashlib.sha256(event_data.encode()).digest()
hash_b32 = bytes.fromhex(h.hex())
print(f'SHA256 hash (hex): {h.hex()}')

# Log hash on-chain (deployer has AUDITOR_ROLE)
try:
    tx = audit_log.functions.logAuditHash(hash_b32, 'permit_application_submitted').build_transaction({
        'from': account.address,
        'gas': 300000,
        'nonce': w3.eth.get_transaction_count(account.address),
        'chainId': w3.eth.chain_id,
    })
    signed = account.sign_transaction(tx)
    tx_hash = w3.eth.send_raw_transaction(signed.raw_transaction)
    receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    if receipt.status == 0:
        print(f'\n⚠ TX REVERTED! Tx: {receipt.transactionHash.hex()}  gasUsed: {receipt.gasUsed}')
        print('  Possible causes: AUDITOR_ROLE missing, hash already exists, or gas too low.')
    else:
        print(f'\nLogged on-chain! Tx: {receipt.transactionHash.hex()}  gasUsed: {receipt.gasUsed}')
except Exception as e:
    if 'Hash already exists' in str(e):
        print('\nHash already on-chain (logged previously).')
    else:
        raise

## 4. Tamper Detection - Verify Hash

Check if a hash exists on-chain. If someone changed the data, the hash would be different — so we'd know it was tampered with.

In [ ]:
# Use SHA256 (same as backend) for bytes32
hash_b32 = bytes.fromhex(h.hex())
exists = audit_log.functions.hashExists(hash_b32).call()
print(f'Hash exists on-chain: {exists}')

# Modified data would produce different hash - tamper detection
tampered_data = 'permit_application_submitted_2024_TAMPERED'
h_tampered = hashlib.sha256(tampered_data.encode()).digest()
exists_tampered = audit_log.functions.hashExists(h_tampered).call()
print(f'Tampered hash exists: {exists_tampered}')

## 5. Security Analysis - onlyOwner / Re-entrancy

Only auditors can log hashes. The contract uses proven patterns to stay secure.

In [ ]:
print('AuditLog uses onlyAuditor modifier - only AUDITOR_ROLE can log.')
print('AccessControl provides role checks. Re-entrancy mitigated by OpenZeppelin patterns.')

## 6. Performance Metrics

Shows the latest block and gas limit. (Basic info about the chain.)

In [ ]:
block = w3.eth.get_block('latest')
print(f'Latest block: {block["number"]}')
print(f'Gas limit: {block["gasLimit"]}')

### 7.1 Prototype Vulnerabilities & Limitations

| Area | Vulnerability / Limitation | Impact | Status |
|------|----------------------------|--------|--------|
| **Single auditor role** | One deployer has AUDITOR_ROLE; no multi-sig or key rotation | Compromised key = attacker can log false hashes | Open |
| **No gas limits on readers** | `getAuditHashCount()` + loop over entries can grow unbounded | DoS if logs grow large; need pagination | Open |
| **Ganache = dev only** | Local chain; no finality, no real decentralization | Not suitable for production; needs mainnet/testnet | Open |
| ~~Private key in notebook~~ | ~~Hardcoded fallback key~~ | ~~Key exposure if notebook shared~~ | **Fixed** — reads from .env only |
| ~~Gradio share link~~ | ~~`share=True` always on~~ | ~~Traffic visible to Gradio servers~~ | **Fixed** — off by default, opt-in via `GRADIO_SHARE=1` |
| ~~No service-to-service auth~~ | ~~Anyone could call audit write endpoints~~ | ~~Unauthorized audit log injection~~ | **Fixed** — `X-API-Key` required on write endpoints |
| ~~No rate limiting~~ | ~~Unbounded writes to blockchain~~ | ~~DoS via rapid log submissions~~ | **Fixed** — rate limiter on `/api/audit/log` |
| ~~Hash-only verification~~ | ~~No way to check if data matches an on-chain hash~~ | ~~Can't verify data integrity from content~~ | **Fixed** — `/verify-data` + Gradio "Verify Data" tab |
| ~~Least privilege~~ | ~~Any user could query any user's audit history~~ | ~~Unauthorized data access~~ | **Fixed** — non-admins see only own logs |

### 7.2 Embedded Prototype UI (Gradio)

View audit logs, verify hashes, and run test scenarios.

In [ ]:
import gradio as gr
import hashlib
import time
import traceback as _tb
from datetime import datetime, timezone

# Rate limiting: max 5 logs per 60 seconds (app-level; contract has no built-in limit)
_RATE_LIMIT_LOGS = []
_RATE_LIMIT_MAX = 5
_RATE_LIMIT_WINDOW = 60

def _check_rate_limit():
    now = time.time()
    _RATE_LIMIT_LOGS[:] = [t for t in _RATE_LIMIT_LOGS if now - t < _RATE_LIMIT_WINDOW]
    if len(_RATE_LIMIT_LOGS) >= _RATE_LIMIT_MAX:
        return False, f'Rate limit: max {_RATE_LIMIT_MAX} logs per {_RATE_LIMIT_WINDOW}s. Try again later.'
    _RATE_LIMIT_LOGS.append(now)
    return True, None

# Debug log buffer — collects messages shown in the Console
_debug_lines = []

def _debug(msg):
    ts = datetime.now(timezone.utc).strftime('%H:%M:%S')
    line = f'[{ts}] {msg}'
    _debug_lines.append(line)
    if len(_debug_lines) > 200:
        del _debug_lines[:len(_debug_lines) - 200]

def _get_debug_log():
    return '\n'.join(_debug_lines) if _debug_lines else '(no debug messages yet)'

def _fetch_audit_logs(limit=50):
    """Fetch audit hash entries from contract into a table."""
    try:
        count = audit_log.functions.getAuditHashCount().call()
        _debug(f'getAuditHashCount() = {count}  (contract: {audit_log.address})')
        rows = []
        for i in range(min(int(count), limit)):
            try:
                entry = audit_log.functions.auditHashEntries(i).call()
                _debug(f'  entry[{i}] raw: {entry}')
                hash_hex = entry[0].hex() if isinstance(entry[0], bytes) else str(entry[0])
                event_type = str(entry[1])
                ts = int(entry[2])
                addr = str(entry[3])
                dt = datetime.fromtimestamp(ts, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S UTC') if ts else ''
                rows.append([str(i + 1), hash_hex, event_type, dt, addr])
            except Exception as inner_e:
                _debug(f'  entry[{i}] ERROR: {inner_e}')
                rows.append([str(i + 1), 'ERROR', str(inner_e), '-', '-'])
        if not rows:
            _debug('No entries found — returning placeholder row')
            return [['—', '(no entries)', '—', '—', '—']]
        _debug(f'Returning {len(rows)} rows')
        return rows
    except Exception as e:
        _debug(f'_fetch_audit_logs EXCEPTION: {e}\n{"".join(_tb.format_exception(e))}')
        return [['!', f'Error: {e}', '—', '—', '—']]

def _table_as_text():
    """Fetch logs and return as tab-separated text for easy copying."""
    rows = _fetch_audit_logs(100)
    lines = ['#\tHash\tEvent Type\tTimestamp\tLogged By']
    for r in rows:
        lines.append('\t'.join(r))
    return '\n'.join(lines)

def verify_hash_ui(hash_hex):
    try:
        h = bytes.fromhex(hash_hex.replace('0x', '').strip())
        if len(h) != 32:
            return 'Invalid hash length (expected 32 bytes / 64 hex chars)'
        exists, ts = audit_log.functions.verifyHash(h).call()
        dt = datetime.fromtimestamp(ts, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S UTC') if ts else '-'
        return f'Exists: {exists}\nTimestamp: {dt}'
    except Exception as e:
        return str(e)

def verify_data_ui(original_data):
    """Verify original data against on-chain hash."""
    if not original_data or not original_data.strip():
        return 'Enter data to verify.'
    try:
        h = hashlib.sha256(original_data.encode()).digest()
        hash_b32 = bytes.fromhex(h.hex())
        exists, ts = audit_log.functions.verifyHash(hash_b32).call()
        dt = datetime.fromtimestamp(ts, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S UTC') if ts else '-'
        if exists:
            return f'Verified: data matches on-chain hash.\nHash: {h.hex()}\nLogged: {dt}'
        return f'Not found: data does not match any stored hash.\nComputed hash: {h.hex()}'
    except Exception as e:
        return str(e)

def log_test_scenario(event_type, event_data):
    """Log a test audit hash. Returns (result, logs, console)."""
    if not event_type or not event_data:
        return 'Enter event type and data.', _fetch_audit_logs(20), _get_debug_log()
    ok, err = _check_rate_limit()
    if not ok:
        return err, _fetch_audit_logs(20), _get_debug_log()
    try:
        h = hashlib.sha256(event_data.encode()).digest()
        hash_b32 = bytes.fromhex(h.hex())
        _debug(f'Logging hash: {h.hex()} eventType: {event_type}')
        _debug(f'  contract: {audit_log.address}  account: {account.address}')
        tx = audit_log.functions.logAuditHash(hash_b32, event_type).build_transaction({
            'from': account.address, 'gas': 300000,
            'nonce': w3.eth.get_transaction_count(account.address), 'chainId': w3.eth.chain_id,
        })
        signed = account.sign_transaction(tx)
        tx_hash = w3.eth.send_raw_transaction(signed.raw_transaction)
        receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
        status = receipt.get('status', 'N/A')
        _debug(f'  tx: {receipt["transactionHash"].hex()}  status: {status}  gasUsed: {receipt.get("gasUsed", "?")}')
        if status == 0:
            _debug('  ⚠ TX REVERTED (status=0) — check AUDITOR_ROLE or hash collision')
        msg = f'Logged! Hash: {h.hex()}\nTx: {receipt["transactionHash"].hex()}\nStatus: {status}'
        return msg, _fetch_audit_logs(20), _get_debug_log()
    except Exception as e:
        _debug(f'log_test_scenario ERROR: {e}\n{"".join(_tb.format_exception(e))}')
        return f'Error: {e}', _fetch_audit_logs(20), _get_debug_log()

def run_test_scenario(scenario):
    """Run predefined test scenarios. Returns (result, logs, console)."""
    if not scenario:
        return 'Select a scenario.', _fetch_audit_logs(20), _get_debug_log()
    _debug(f'Running scenario: {scenario}')
    if scenario == 'Log sample permit':
        return log_test_scenario('permit_application_submitted', 'permit_sample_2024')
    if scenario == 'Verify known hash':
        result = verify_hash_ui('7f64fe1375f7dbda9f4d2b1973015c0db8f60753f621067e29fd54c961ce46f4')
        return result, _fetch_audit_logs(20), _get_debug_log()
    if scenario == 'Verify known data':
        result = verify_data_ui('permit_application_submitted_2024')
        return result, _fetch_audit_logs(20), _get_debug_log()
    if scenario == 'Tamper check':
        result = verify_hash_ui(hashlib.sha256(b'permit_TAMPERED').hex()) + '\n(Expected: false — hash not logged)'
        return result, _fetch_audit_logs(20), _get_debug_log()
    return 'Unknown scenario.', _fetch_audit_logs(20), _get_debug_log()

def _startup_load():
    _debug(f'Startup — contract: {audit_log.address}  account: {account.address}')
    _debug(f'  balance: {w3.from_wei(w3.eth.get_balance(account.address), "ether"):.2f} ETH')
    return _fetch_audit_logs(20), _get_debug_log()

def _refresh_all():
    return _fetch_audit_logs(20), _get_debug_log()

# ─── Gradio UI ───────────────────────────────────────────────────────────────
with gr.Blocks(title='BizClear Blockchain Audit Prototype', theme=gr.themes.Soft()) as demo:
    gr.Markdown('# BizClear Blockchain Audit Prototype')
    gr.Markdown('View logs, verify hashes, and run test scenarios.  \n'
                '*Tip: click any text field → Ctrl+A → Ctrl+C to copy.*')

    with gr.Tabs():
        with gr.TabItem('View Audit Logs'):
            logs_table = gr.Dataframe(
                headers=['#', 'Hash', 'Event Type', 'Timestamp', 'Logged By'],
                datatype=['str', 'str', 'str', 'str', 'str'],
                label='Audit Hash Logs (on-chain)',
                wrap=True,
            )
            refresh_btn = gr.Button('Refresh Logs')
            export_btn = gr.Button('Export as Text')
            export_out = gr.Textbox(label='Exported Table (select all → copy)', lines=6, interactive=True)

        with gr.TabItem('Verify Hash'):
            hash_in = gr.Textbox(label='SHA256 Hash (hex, 64 chars)', placeholder='7f64fe13...')
            verify_out = gr.Textbox(label='Result', lines=3, interactive=True)
            verify_hash_btn = gr.Button('Verify')

        with gr.TabItem('Verify Data'):
            data_in = gr.Textbox(label='Original data (content to verify)', placeholder='permit_application_submitted_2024', lines=3)
            data_verify_out = gr.Textbox(label='Result', lines=3, interactive=True)
            verify_data_btn = gr.Button('Verify')

        with gr.TabItem('Log Test Event'):
            evt_type = gr.Textbox(label='Event Type', value='permit_application_submitted')
            evt_data = gr.Textbox(label='Event Data (will be hashed)', value='test_permit_2024')
            log_out = gr.Textbox(label='Result', lines=4, interactive=True)
            log_btn = gr.Button('Log on-chain')

        with gr.TabItem('Test Scenarios'):
            scenario_dd = gr.Dropdown(
                choices=['Log sample permit', 'Verify known hash', 'Verify known data', 'Tamper check'],
                label='Scenario',
            )
            scenario_out = gr.Textbox(label='Result', lines=4, interactive=True)
            run_btn = gr.Button('Run')

    gr.Markdown('### Console')
    console_box = gr.Textbox(label='Debug Log (errors, contract calls, tx status)', lines=12, interactive=True)
    refresh_console_btn = gr.Button('Refresh Console')

    # Wire up events
    refresh_btn.click(fn=_refresh_all, outputs=[logs_table, console_box])
    export_btn.click(fn=_table_as_text, outputs=export_out)
    verify_hash_btn.click(fn=verify_hash_ui, inputs=hash_in, outputs=verify_out)
    verify_data_btn.click(fn=verify_data_ui, inputs=data_in, outputs=data_verify_out)
    log_btn.click(fn=log_test_scenario, inputs=[evt_type, evt_data], outputs=[log_out, logs_table, console_box])
    run_btn.click(fn=run_test_scenario, inputs=scenario_dd, outputs=[scenario_out, logs_table, console_box])
    refresh_console_btn.click(fn=_get_debug_log, outputs=console_box)
    demo.load(fn=_startup_load, outputs=[logs_table, console_box])

_share = os.getenv('GRADIO_SHARE', '').strip() in ('1', 'true', 'True')
demo.launch(share=_share)